In [ ]:
%pip install lightgbm
%pip install rich

In [ ]:
from pathlib import Path
import json

from rich import print
import numpy as np
import pandas as pd
import sagemaker
from sagemaker import Model
import tarfile
import boto3
import lightgbm as lgb

## Model Test

In [3]:
model = lgb.Booster(model_file='model.lgb')

instances = [
  {
    "bill_length_mm": 39.1,
    "bill_depth_mm": 18.7,
    "flipper_length_mm": 181,
    "body_mass_g": 3750
  },
  {
    "bill_length_mm": 46.5,
    "bill_depth_mm": 17.9,
    "flipper_length_mm": 192,
    "body_mass_g": 3500
  },
  {
    "bill_length_mm": 46.1,
    "bill_depth_mm": 13.2,
    "flipper_length_mm": 211,
    "body_mass_g": 4500
  }
]

df = pd.DataFrame(instances)
display(df)

preds = model.predict(df)
print(preds)


,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,39.1,18.7,181,3750
1,46.5,17.9,192,3500
2,46.1,13.2,211,4500


[[0.99291988 0.00489147 0.00218865]
 [0.01591752 0.97967967 0.00440281]
 [0.00564193 0.00662217 0.9877359 ]]

## モデルをS3にアップロード

In [4]:
with tarfile.open('model.tar.gz', 'w:gz') as f:
    f.add('model.lgb')

In [ ]:
!aws s3 cp model.tar.gz {Specify the S3_URI here}

## ECRにコンテナをアップロード

In [6]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
ecr_repository = 'sagemaker/lightgbm'
image_tag = 'latest'
image_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{ecr_repository}:{image_tag}'

In [ ]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)

# ECRリポジトリの作成
!aws ecr create-repository --repository-name $ecr_repository

# コンテナイメージビルド
!docker build -t $image_uri .

# ECRにpush
!docker push $image_uri

## SageMaker にデプロイ

instance type:
ml.r7i.48xlarge, ml.trn1.32xlarge, ml.r5d.12xlarge, ml.r5.12xlarge, ml.p2.xlarge, ml.m5.4xlarge, ml.m4.16xlarge, ml.r7i.16xlarge, ml.m7i.xlarge, ml.p5.48xlarge, ml.r6gd.xlarge, ml.r6g.8xlarge, ml.r6g.large, ml.m6gd.16xlarge, ml.r5d.24xlarge, ml.r5.24xlarge, ml.r7i.8xlarge, ml.r7i.large, ml.m7i.12xlarge, ml.r6gd.12xlarge, ml.r6g.16xlarge, ml.m6gd.8xlarge, ml.m6gd.large, ml.m6g.xlarge, ml.p4d.24xlarge, ml.m7i.24xlarge, ml.m6g.12xlarge, ml.g5.2xlarge, ml.p3.16xlarge, ml.m5d.xlarge, ml.m5.large, ml.t2.xlarge, ml.m7i.48xlarge, ml.p2.16xlarge, ml.m5d.12xlarge, ml.m7i.16xlarge, ml.r6gd.16xlarge, ml.c6gd.2xlarge, ml.g5.4xlarge, ml.inf1.2xlarge, ml.m5d.24xlarge, ml.m6g.16xlarge, ml.c4.2xlarge, ml.c6gn.xlarge, ml.c6gd.4xlarge, ml.c5.2xlarge, ml.c6gn.12xlarge, ml.c6i.32xlarge, ml.c4.4xlarge, ml.g5.8xlarge, ml.c6i.xlarge, ml.inf1.6xlarge, ml.c5d.2xlarge, ml.c5.4xlarge, ml.c7i.xlarge, ml.c7g.2xlarge, ml.c6i.12xlarge, ml.g4dn.xlarge, ml.c7i.12xlarge, ml.c6gd.8xlarge, ml.c6gd.large, ml.c6g.2xlarge, ml.c6g.xlarge, ml.c6i.24xlarge, ml.g4dn.12xlarge, ml.c5d.4xlarge, ml.c7i.24xlarge, ml.c7i.2xlarge, ml.inf2.8xlarge, ml.c6gn.16xlarge, ml.c6g.12xlarge, ml.c7g.4xlarge, ml.c7g.xlarge, ml.g4dn.2xlarge, ml.c4.8xlarge, ml.c4.large, ml.c6g.4xlarge, ml.c7g.12xlarge, ml.c6i.2xlarge, ml.c5d.xlarge, ml.c5.large, ml.c7i.48xlarge, ml.c7i.4xlarge, ml.c6i.16xlarge, ml.g4dn.4xlarge, ml.c5.9xlarge, ml.c7i.16xlarge, ml.c6gn.2xlarge, ml.c6i.4xlarge, ml.g4dn.16xlarge, ml.c5d.large, ml.c5.xlarge, ml.inf2.xlarge, ml.c6g.16xlarge, ml.c7g.8xlarge, ml.c7g.large, ml.c5d.9xlarge, ml.c4.xlarge, ml.trn1n.32xlarge, ml.c6gn.4xlarge, ml.c6gd.xlarge, ml.c6g.8xlarge, ml.c6g.large, ml.c7g.16xlarge, ml.inf1.xlarge, ml.c7i.8xlarge, ml.c7i.large, ml.inf2.24xlarge, ml.c6gd.12xlarge, ml.g4dn.8xlarge, ml.g5.xlarge, ml.c6i.8xlarge, ml.c6i.large, ml.inf1.24xlarge, ml.m5d.2xlarge, ml.t2.2xlarge, ml.inf2.48xlarge, ml.g5.12xlarge, ml.c5d.18xlarge, ml.c6gn.8xlarge, ml.c6gn.large, ml.m6g.2xlarge, ml.g5.24xlarge, ml.m5d.4xlarge, ml.t2.medium, ml.m7i.2xlarge, ml.trn1.2xlarge, ml.r6gd.2xlarge, ml.c6gd.16xlarge, ml.c5.18xlarge, ml.m6g.4xlarge, ml.g5.48xlarge, ml.m7i.4xlarge, ml.r6gd.4xlarge, ml.g5.16xlarge, ml.dl1.24xlarge, ml.r5d.2xlarge, ml.r5.2xlarge, ml.p3.2xlarge, ml.m5d.large, ml.m5.xlarge, ml.m4.10xlarge, ml.t2.large, ml.r6g.2xlarge, ml.r5d.4xlarge, ml.r5.4xlarge, ml.m5.12xlarge, ml.m4.xlarge, ml.r7i.2xlarge, ml.r7i.xlarge, ml.m6gd.2xlarge, ml.m6gd.xlarge, ml.m6g.8xlarge, ml.m6g.large, ml.m5.24xlarge, ml.r7i.12xlarge, ml.m7i.8xlarge, ml.m7i.large, ml.r6gd.8xlarge, ml.r6gd.large, ml.r6g.4xlarge, ml.r6g.xlarge, ml.m6gd.12xlarge, ml.m4.2xlarge, ml.r7i.24xlarge, ml.r7i.4xlarge, ml.r6g.12xlarge, ml.m6gd.4xlarge, ml.p2.8xlarge, ml.m5.2xlarge, ml.p4de.24xlarge, ml.r5d.xlarge, ml.r5d.large, ml.r5.xlarge, ml.r5.large, ml.p3.8xlarge, ml.m4.4xlarge

In [14]:
MODEL_URI = '{Specify the S3_URI here}/model.tar.gz'


model = Model(
    image_uri=image_uri,
    model_data=MODEL_URI,
    entry_point='entry_point.py',
    source_dir='./code',
    role=sagemaker.get_execution_role(),
    predictor_cls=sagemaker.predictor.Predictor,
)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.large')


---!

## エンドポイントのテスト

In [15]:
# 推論したいデータを準備
instances = [
  {
    "bill_length_mm": 39.1,
    "bill_depth_mm": 18.7,
    "flipper_length_mm": 181,
    "body_mass_g": 3750
  },
  {
    "bill_length_mm": 46.5,
    "bill_depth_mm": 17.9,
    "flipper_length_mm": 192,
    "body_mass_g": 3500
  },
  {
    "bill_length_mm": 46.1,
    "bill_depth_mm": 13.2,
    "flipper_length_mm": 211,
    "body_mass_g": 4500
  }
]
payload = json.dumps(instances)

result = predictor.predict(payload, initial_args={'ContentType': 'application/json'})
print(result.decode())

[[0.9929198838092113, 0.0048914660096075575, 0.00218865018118128], [0.015917523677293766, 0.9796796692112346, 
0.0044028071114715584], [0.005641933018067216, 0.006622168832683619, 0.9877358981492491]]

In [16]:
client = boto3.client('sagemaker-runtime')


# 推論したいデータを準備
instances = [
  {
    "bill_length_mm": 39.1,
    "bill_depth_mm": 18.7,
    "flipper_length_mm": 181,
    "body_mass_g": 3750
  },
  {
    "bill_length_mm": 46.5,
    "bill_depth_mm": 17.9,
    "flipper_length_mm": 192,
    "body_mass_g": 3500
  },
  {
    "bill_length_mm": 46.1,
    "bill_depth_mm": 13.2,
    "flipper_length_mm": 211,
    "body_mass_g": 4500
  }
]
payload = json.dumps(instances).encode('utf-8')

endpoint_name = predictor.endpoint_name
print(endpoint_name)

# 推論リクエストを送信
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType='application/json',  # コンテンツタイプを指定
)

# レスポンスから推論結果を取得
result = json.loads(response['Body'].read().decode())
print(result)



lightgbm-2024-03-02-11-02-26-540

[
    [0.9929198838092113, 0.0048914660096075575, 0.00218865018118128],
    [0.015917523677293766, 0.9796796692112346, 0.0044028071114715584],
    [0.005641933018067216, 0.006622168832683619, 0.9877358981492491]
]

## エンドポイントの削除

In [17]:
client = boto3.client('sagemaker')

endpoint_description = client.describe_endpoint(EndpointName=endpoint_name)
endpoint_config_name = endpoint_description['EndpointConfigName']
endpoint_config_description = client.describe_endpoint_config(EndpointConfigName=endpoint_config_name)
model_names = [variant['ModelName'] for variant in endpoint_config_description['ProductionVariants']]

# デプロイで作成されたリソースを削除
client.delete_endpoint(EndpointName=endpoint_name)
client.delete_endpoint_config(EndpointConfigName=endpoint_description['EndpointConfigName'])
for model_name in model_names:  # モデル名を取得, エンドポイント設定には複数のモデルを含むことができるため、ProductionVariants をループしてモデル名を取得する
    client.delete_model(ModelName=model_name)